In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from time import strptime
import datetime
from datetime import datetime
# from statistics import mean
import warnings
import time
import csv
import glob
import logging
import importlib
import json
from tqdm.notebook import tqdm
import sys
sys.path.append('C:/Users/gianl/Desktop/Gi/Supsi/BachelorProject/notebook/tweet_utils')
import tweets_utils
# import networkx as nx
# import seaborn as sns
# import re
# from operator import itemgetter
# import matplotlib.cm as mcm
# import community
# from community import community_louvain as cm
# import leidenalg as la
# import igraph as ig
# from cdlib import algorithms
import os
warnings.filterwarnings('ignore')
# %autosave 3600

In [2]:
# Change data path and file names accordingly
data_path = "F:/Supsi/COVID-19-Tweets/large_files/rt_network/names/"
network_path = os.path.join(data_path, "user_rtuser.csv")
disinform_path = "C:/Users/gianl/Desktop/Gi/Supsi/BachelorProject/csv/disinformation_users.csv"
good_path = "C:/Users/gianl/Desktop/Gi/Supsi/BachelorProject/csv/all_good_users.csv"
disinform = pd.read_csv(disinform_path)
good = pd.read_csv(good_path)

In [3]:
good_lst = list(good["screen_name"])
disinform_lst = list(disinform["screen_name"])

In [4]:
disinform = disinform[["screen_name"]]


In [3]:
# network_df = pd.DataFrame()
# for chunk in pd.read_csv(network_path, chunksize=100000):
#         network_df = chunk
#         break
network_df = pd.read_csv(network_path)
network_df

,name,rt_name
0,Huerconetzin,AnneKPIX
1,JustAnotherAme4,cnni
2,HHSRegion8,CDCgov
3,Paxman42,OurWarOnCancer
4,beerhowell,nytimes
...,...,...
48312499,Dinora21971394,saludhn
48312500,EllyJonkerVliet,StopEcocideNL
48312501,anna76746180,Haggis_UK
48312502,SueHealy16,Auchindrain


In [56]:
retweets_graph = nx.from_pandas_edgelist(network_df,'name','rt_name', create_using=nx.DiGraph())
print(nx.info(retweets_graph))

DiGraph with 68949 nodes and 90606 edges


In [57]:
in_deg_centrality = nx.in_degree_centrality(retweets_graph)
out_deg_centrality = nx.out_degree_centrality(retweets_graph)

In [59]:
out_good = []
out_bad = []
in_good = []
in_bad = []
for i in tqdm(out_deg_centrality):
    if i in good_lst: 
        out_good.append(out_deg_centrality.get(i))
    elif i in disinform_lst:
        out_bad.append(out_deg_centrality.get(i))
        
for i in tqdm(in_deg_centrality):
    if i in good_lst: 
        in_good.append(in_deg_centrality.get(i))
    elif i in disinform_lst:
        in_bad.append(in_deg_centrality.get(i))
        
print(f"Good in: {mean(in_good)}")
print(f"Bad in: {mean(in_bad)}")
print(f"Good out: {mean(out_good)}")
print(f"Bad out: {mean(out_bad)}")


Good in: 0.00016906202572399955
Bad in: 7.251841967859836e-05
Good out: 3.520741596609814e-05
Bad out: 1.4503683935719673e-05


In [64]:
retweets_graph.remove_edges_from(nx.selfloop_edges(retweets_graph))
k_core = nx.algorithms.k_core(retweets_graph)
cores = nx.algorithms.core.core_number(k_core)
cores

In [16]:
rt = chunk.merge(good, left_on="rt_name", right_on="screen_name", how="left")
rt[rt["screen_name"].notna()]

,name,rt_name,screen_name
59,billm9,ezraklein,ezraklein
133,codename_karla,ezraklein,ezraklein
268,OhioDistrict12,ezraklein,ezraklein
309,MCUfeld,ezraklein,ezraklein
370,jasr9203,ezraklein,ezraklein
...,...,...,...
999339,herrheath,maggieNYT,maggieNYT
999366,MoreBooks4All,maggieNYT,maggieNYT
999700,NancyBonita7,maggieNYT,maggieNYT
999891,coffeytawk2,maggieNYT,maggieNYT


In [19]:
network_df = pd.DataFrame()
rt_good_lst = [] 
rt_bad_lst = [] 
for chunk in tqdm(pd.read_csv(network_path, chunksize=1000000, lineterminator="\n", low_memory=False)):
        print("chunk processing... ")
        rt = chunk.merge(good, left_on="rt_name", right_on="screen_name", how="left")
        rt_good_lst.extend(list(pd.Series(rt[rt["screen_name"].notna()]["name"]).unique()))
        rt = chunk.merge(disinform, left_on="rt_name", right_on="screen_name", how="left")
        rt_bad_lst.extend(list(pd.Series(rt[rt["screen_name"].notna()]["name"]).unique()))
        print("Done!")

chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
chunk processing... 
Done!
c

In [23]:
good_rt = list(set(rt_good_lst))
bad_rt = list(set(rt_bad_lst))
pd.DataFrame(good_rt, columns=["screen_name"]).to_csv("C:/Users/gianl/Desktop/Gi/good_rt.csv", line_terminator="\n",
                                                     encoding="utf-8", index=False)
pd.DataFrame(bad_rt, columns=["screen_name"]).to_csv("C:/Users/gianl/Desktop/Gi/bad_rt.csv", line_terminator="\n",
                                                     encoding="utf-8", index=False)


In [ ]:
rt_bad = network_df.merge(disinform, left_on="name", right_on="screen_name", how="left")
rt_bad[rt_bad["rt_name"].notna()]

In [75]:
# network_df

In [74]:
# nt_1 = list(pd.read_csv("F:/Supsi/COVID-19-Tweets/large_files/rt_network/names/rt_nt_1.csv")["name"])
# nt_2 = list(pd.read_csv("F:/Supsi/COVID-19-Tweets/large_files/rt_network/names/rt_nt_2.csv")["name"])


In [73]:
# for k,v in tqdm(network_df.iterrows()):
#     if (v["name"] in nt_1) & (v["rt_name"] in nt_2):
#         print(v)